In [16]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()

In [17]:
num_features = 2
num_samples = 5
state_size = 4
num_features = 4
num_of_operations = 3
batch_size = num_samples
max_num_features = 10
datatype = tf.float64


#model placeholders
batchX_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchX")
batchY_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchY")

def add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((-1,1))

    #x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    #y = y.reshape((batch_size, -1))

    return (x, y)


x,y = samples_generator(add, (num_samples, num_features) , (0, 100))

def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])

def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a


x = tf.Variable(x, dtype=datatype)

x.initializer.run()



In [18]:
add    = tf_add(x)
mult   = tf_multiply(x)
stall  = tf_stall(x)
values = tf.concat([add, mult, stall], 1)
print(values.eval())

[[  2.59408824e+02   1.48458952e+07   8.29336205e+01   4.18525764e+01
    5.13842322e+01   8.32383946e+01]
 [  2.01165266e+02   4.37527359e+06   8.75524478e+01   2.67414882e+01
    3.92040709e+01   4.76672595e+01]
 [  1.94187721e+02   7.21788032e+05   9.67716182e+01   1.18416942e+01
    8.13349831e+00   7.74409103e+01]
 [  1.41914639e+02   8.64847580e+05   3.81070336e+01   6.31182577e+01
    1.29731689e+01   2.77161789e+01]
 [  3.15773540e+02   3.70523096e+07   8.45986044e+01   9.61823107e+01
    6.89561711e+01   6.60364534e+01]]


In [19]:
init_state = tf.Variable(np.zeros([batch_size, state_size]), dtype=datatype)
argmax_dum = tf.Variable([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  dtype=datatype)
W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype)
b = tf.Variable(np.zeros((state_size)), dtype=datatype)

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype)
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=datatype)

dummy_matrix = tf.zeros([batch_size, num_features], datatype, name="dummy_constant")

init_state.initializer.run()
argmax_dum.initializer.run()
W.initializer.run()
b.initializer.run()
W2.initializer.run()
b2.initializer.run()
#dummy_matrix.initializer.run()

current_state = init_state
current_input = x
      
input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
current_state = next_state
    
    #calculate softmax and produce the mask of operations
logits  = tf.matmul(next_state, W2) + b2 #Broadcasted addition
softmax = tf.nn.softmax(logits)
softmax_10000 = tf.nn.softmax(logits)
softplus = tf.nn.softplus(logits)
softsign = tf.nn.softsign(logits)
argmax  = tf.argmax(softmax, 1)

#lengths_transposed = tf.expand_dims(argmax, 0)
#onehot  = tf.one_hot(argmax, num_of_operations)
stall_width = tf.shape(stall)[1] - 1 
stall_select = tf.slice(softmax, [0,2], [batch_size,1])
multiples = [1, stall_width]
stall_mask = tf.tile(stall_select, multiples)
mask = tf.concat([softmax, stall_mask], 1)
#mask = tf.cast(mask, tf.bool)
#apply mask
#output = tf.boolean_mask(values,mask)
output = values * mask
nn_soft = tf.nn.softmax(logits)
output_width = tf.shape(output)[1]
zeros_slice = tf.slice(dummy_matrix, [0,0], [batch_size, max_num_features - output_width])
dum_plus_input = tf.concat([output, zeros_slice], 1)
print(dum_plus_input.eval())

'''
print(x.eval())
print("stall_select")
print(stall_select.eval())
print("mask")
print(mask.eval())
print("logits")
print(logits.eval())
print("softmax")
print(softmax.eval())
print("softplus")
print(softplus.eval())
print("softsign")
print(softsign.eval())
print("softmax*10000")
print(softmax_10000.eval())
print("onehot")
print(onehot.eval())
'''
print(mask.eval())
#print(mask.eval())
#print(output.eval())

[[  1.25249222e+02   4.06975727e+06   2.01562609e+01   1.01718874e+01
    1.24884695e+01   2.02303335e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.71277415e+01   1.19940908e+06   2.12788249e+01   6.49927509e+00
    9.52819228e+00   1.15850932e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.37588040e+01   1.97866283e+05   2.35194602e+01   2.87801591e+00
    1.97677267e+00   1.88213078e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.85200216e+01   2.37083698e+05   9.26156736e+00   1.53403175e+01
    3.15301052e+00   6.73616480e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.52463550e+02   1.01572795e+07   2.05609201e+01   2.33762344e+01
    1.67591692e+01   1.60495584e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
[[ 0.48282561  0.2741335   0.24304089  0.24304089  0.24304089  0.24304089]
 [ 0.48282561  0.2741335   0.24304089  

In [20]:
add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]


add_final   = add * add_softmax
mult_final  = mult * mult_softmax
stall_final = stall * stall_softmax

##conact add and mult results with zeros matrix
add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

output_final = add_final + mult_final + stall_final 

print(add_width.eval())
print(mult_width.eval())
print(stall_width.eval())


print(add.eval())
print(mult.eval())
print(stall.eval())

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

print(output_final.eval())

1
1
4
[[ 259.40882367]
 [ 201.16526641]
 [ 194.18772105]
 [ 141.91463916]
 [ 315.77353959]]
[[ 14845895.18126184]
 [  4375273.58868517]
 [   721788.03170229]
 [   864847.57969874]
 [ 37052309.60385395]]
[[ 82.93362049  41.85257639  51.38423224  83.23839455]
 [ 87.55244785  26.74148817  39.20407087  47.66725952]
 [ 96.7716182   11.84169424   8.13349831  77.4409103 ]
 [ 38.10703357  63.11825774  12.97316892  27.71617892]
 [ 84.59860445  96.18231067  68.95617112  66.03645336]]
[[ 0.48282561]
 [ 0.48282561]
 [ 0.48282561]
 [ 0.48282561]
 [ 0.48282561]]
[[ 0.2741335]
 [ 0.2741335]
 [ 0.2741335]
 [ 0.2741335]
 [ 0.2741335]]
[[ 0.24304089]
 [ 0.24304089]
 [ 0.24304089]
 [ 0.24304089]
 [ 0.24304089]]
[[ 125.24922232    0.            0.            0.        ]
 [  97.12774153    0.            0.            0.        ]
 [  93.75880396    0.            0.            0.        ]
 [  68.52002156    0.            0.            0.        ]
 [ 152.4635504     0.            0.            0.        ]]
[[

In [21]:

def custom_softmax(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = tf.log(sum_exponents)
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( inpt / soft_max, [batch_size, -1])

def custom_softmax2(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = sum_exponents
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( tf.exp(inpt) / soft_max, [batch_size, -1])
x_cust = custom_softmax(x)
x_cust2 = custom_softmax2(x)
print(x.eval())
#print(x_cust2.eval())
print(tf.nn.softmax(x*10000).eval())

[[ 83.79072094]
 [ 87.55244785]
 [ 96.7716182 ]
 [ 63.11825774]
 [ 96.18231998]]
[[  2.45384846e+36]
 [  1.05571083e+38]
 [  1.06507402e+42]
 [  2.58173133e+27]
 [  5.90814065e+41]]
[[ 82.93362049  41.85257639  51.38423224  83.23839455]
 [ 87.55244785  26.74148817  39.20407087  47.66725952]
 [ 96.7716182   11.84169424   8.13349831  77.4409103 ]
 [ 38.10703357  63.11825774  12.97316892  27.71617892]
 [ 84.59860445  96.18231067  68.95617112  66.03645336]]
[[ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]]


In [22]:
math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(tf.reduce_sum(output, axis = 1, name="red_output") , batchY_placeholder, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")

total_loss = tf.reduce_sum(math_error, name="red_total_loss")

grads = tf.gradients(total_loss, [W,b,W2,b2], name="comp_gradients")
train_step = tf.train.AdamOptimizer(0.1, name="AdamOpt").minimize(total_loss, name="min_loss")

In [27]:
grads[2].eval()

InvalidArgumentError: You must feed a value for placeholder tensor 'batchY_2' with dtype double and shape [5,?]
	 [[Node: batchY_2 = Placeholder[dtype=DT_DOUBLE, shape=[5,?], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'batchY_2', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.4/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.4/dist-packages/

In [15]:
sess.close()